In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import pickle
import os
import glob

In [3]:
df_market = pd.read_parquet('../processed_market_data.parquet')
df_market.columns

Index(['date', 'order_book_id', 'adj_close', 'adj_factor', 'adj_high',
       'adj_low', 'adj_open', 'circulation_a', 'circulation_market_value',
       'close', 'halt_status', 'high', 'limit_down', 'limit_up', 'low',
       'num_trades', 'open', 'st_status', 'total_a', 'total_turnover',
       'turnover_rate', 'volume', 'st_change', 'halt_change', 'day_count',
       'ret_overnight', 'ret_day', 'ret_daily'],
      dtype='object')

In [4]:
directory = "Z:/1_min_data"

all_files = glob.glob(directory + "/*.pkl")
data_frames = {}
for file in all_files:
    df_1min_day = pd.read_pickle(file)
    file_name = os.path.basename(file).split('.')[0]  

    df_1min_day.loc[df_1min_day['TIME'] <= 36000, 'Time_seg'] = '1'
    df_1min_day.loc[(df_1min_day['TIME'] < 52200) & (df_1min_day['TIME'] > 36000), 'Time_seg'] = '2'
    df_1min_day.loc[df_1min_day['TIME'] >= 52200, 'Time_seg'] = '3'

    df_1min_day = pd.merge(df_1min_day,df_market[['date','order_book_id','circulation_a']],left_on=['DATE','RIC'],right_on=['date','order_book_id'],how='left')
    df_1min_day['min_turnover'] = df_1min_day['VOL']/df_1min_day['circulation_a']

    df_day = df_1min_day.groupby(['order_book_id','Time_seg'])['min_turnover'].std().reset_index()
    # df_day = df_1min_day.groupby(['order_book_id'])['min_turnover'].std().reset_index()


    data_frames[file_name] = df_day


In [5]:
for date, df in data_frames.items():
    df['date'] = date

df = pd.concat(data_frames.values(),axis=0)

In [6]:
df = df.rename(columns={'min_turnover':'TurnVol_daily'})
df

,order_book_id,Time_seg,TurnVol_daily,date
0,000001,1,0.000012,2016-01-04
1,000001,2,0.000028,2016-01-04
2,000001,3,0.000000,2016-01-04
3,000004,1,0.000326,2016-01-04
4,000004,2,0.000208,2016-01-04
...,...,...,...,...
14896,688981,2,0.000121,2024-11-18
14897,688981,3,0.000081,2024-11-18
14898,689009,1,0.000801,2024-11-18
14899,689009,2,0.000544,2024-11-18


In [7]:
# df.to_parquet('daily_turnover_volatility.parquet')
df.to_parquet('daily_turnover_vol_timesegment.parquet')